# CNN Model

In [ ]:
import numpy as np
X = np.load('../SignLanguage/X.npy')
Y = np.load('../SignLanguage/Y.npy')

In [ ]:
def plot_digits_colidx(X, Y):
    plt.figure(figsize=(10,10))
    plt.plot([5, 2, 11])
    for i in col_idx:
        ax = plt.subplot(5, 2, i+1)
        ax.set_title("Column_idx: " + str(i))
        plt.axis('off')
        plt.imshow(X[np.argwhere(Y[:,i]==1)[0][0],:], cmap='gray')

In [ ]:
import matplotlib.pyplot as plt
N_classes = Y.shape[1]
col_idx = [i for i in range(N_classes)]
plot_digits_colidx(X, Y)

In [ ]:
#dictionary that handles the column index - digit relatinship
colidx_digit = {0: 9,
                1: 0,
                2: 7,
                3: 6,
                4: 1,
                5: 8,
                6: 4,
                7: 3,
                8: 2,
                9: 5}

#digit - column index relationship dictionary
digit_colidx = {v: k for k, v in colidx_digit.items()}

In [ ]:
#create empty matrix
Y_ordered = np.zeros(Y.shape)
#fill the matrix so that the columns index also corresponds to the digit
for i in range(N_classes):
    Y_ordered[:, i] = Y[:, digit_colidx[i]]

In [ ]:
plot_digits_colidx(X, Y_ordered)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_ordered, test_size=0.3, random_state=42)

In [ ]:
X_train.shape

In [ ]:
import numpy as np
# Reshape the images.
train_images = np.expand_dims(X_train, axis=3)
val_images = np.expand_dims(X_test, axis=3)

print(train_images.shape) # (60000, 28, 28, 1)
print(val_images.shape)  # (10000, 28, 28, 1)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D, Conv2D, MaxPooling2D

seed = 7
np.random.seed(seed)
tf.set_random_seed(seed)
#tf.random.set_seed(seed)

model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=8, kernel_size=(3,3),activation='relu')) #82% avec 16
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))
#model.add(MaxPooling2D(pool_size=(4,4)))
#model.add(Dense(128, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [ ]:
import keras
# Set Optimizer
#opt = adam(lr=0.001)
model.compile(
  loss='categorical_crossentropy',
  optimizer=keras.optimizers.Adamax(),
  metrics=['accuracy'],
)

In [ ]:
# ignore Warning
import warnings
warnings.filterwarnings("ignore")

history = model.fit(
  train_images,
  y_train,
  epochs=30,
  validation_data=(val_images, y_test),
  batch_size=X_train.shape[1]
)

In [ ]:
model.summary()

In [ ]:
# Predict.
predictions = model.predict(val_images)

# Print our model's predictions.
print(np.argmax(predictions, axis=1))

# Check our predictions against the ground truths.
print(np.argmax(y_test, axis=1)) 

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score

y_train_class = y_test.argmax(axis=1)
y_pred_class = predictions.argmax(axis=1)
conf_mat = confusion_matrix(y_train_class, y_pred_class)
plt.figure(figsize=(15,10))
sns.heatmap(conf_mat,square=True, annot=True, fmt='.0f')
plt.show()
print("accuracy :" + str(accuracy_score(y_train_class, y_pred_class)*100)+" %")

# Pre-trained CNN

In [ ]:
X_train_vide = np.empty([train_images.shape[0], train_images.shape[1], train_images.shape[2], 3])
X_train_vide[..., :] = train_images[..., :]
train_images = X_train_vide
train_images[10,6,5,:]

In [ ]:
X_val_vide = np.empty([val_images.shape[0], val_images.shape[1], val_images.shape[2], 3])
X_val_vide[..., :] = val_images[..., :]
val_images = X_val_vide
val_images[10,6,5,:]

In [ ]:
from keras.applications.mobilenet import MobileNet
model = MobileNet()
print(model.summary())

In [ ]:
from keras.applications import MobileNet
import keras
from keras.optimizers import SGD, Adamax, Adadelta, Adam       
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping

seed = 7
np.random.seed(seed)
tf.set_random_seed(seed)
#tf.random.set_seed(seed)

callbacks = [EarlyStopping(monitor='val_accuracy', patience=10)]

pretrained_model = MobileNet(include_top=False, input_shape=(64, 64, 3), weights='imagenet')
#for layer in pretrained_model.layers:
    #layer.trainable = False

model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adamax',    
              metrics=['accuracy'])

In [ ]:
# Train
history = model.fit(train_images, y_train, batch_size=train_images.shape[0], epochs=20,validation_data=(val_images,y_test),callbacks=callbacks)

In [ ]:
model.summary()

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Predict.
predictions = model.predict(val_images)

# Print our model's predictions.
print(np.argmax(predictions, axis=1))

# Check our predictions against the ground truths.
print(np.argmax(y_test, axis=1)) 

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score

y_train_class = y_test.argmax(axis=1)
y_pred_class = predictions.argmax(axis=1)
conf_mat = confusion_matrix(y_train_class, y_pred_class)
plt.figure(figsize=(15,10))
sns.heatmap(conf_mat,square=True, annot=True, fmt='.0f')
plt.show()
print("accuracy :" + str(accuracy_score(y_train_class, y_pred_class)*100)+" %")